In [1]:
import numpy as np
import pandas as pd
import os
import nltk
from nltk.util import ngrams
from nltk import word_tokenize
from collections import Counter
import re
import string
from nltk.corpus import stopwords
import sys
from nltk.stem import PorterStemmer

def feature_noun(pos_tag):
    if pos_tag[1] == 'NN' or pos_tag[1] =='NNP':
        return 1
    else:
        return 0
    
def feature_adjective(pos_tag):
    if pos_tag[1] == 'JJ':
        return 1
    else:
        return 0
    
def feature_verb(pos_tag):
    if pos_tag[1] == 'VB' or pos_tag[1] == 'VBD' or pos_tag[1] == 'VBG' or pos_tag[1] == 'VBN':
        return 1
    else:
        return 0

def end_sent(idx):
    if idx == 0:
        return 1
    elif idx != 0 and idx-3 == 0:
        return 1
    else:
        return 0
    
def feat_specific(dest, gram):
    
    ps = PorterStemmer()
 
    if len(gram) == 2:
        if gram[0].lower()  in dest or gram[1].lower()  in dest:
            return 1
        else:
            return 0
    elif len(gram) == 3:
        if gram[0].lower() in dest or gram[1].lower()  in dest or gram[2].lower()  in dest:
            return 1
        else:
            return 0
    else:
        if gram.lower() in dest:
            return 1
        else:
            return 0

def num_capitals(gram):
    string = ' '.join(gram)
    return sum(1 for c in string if c.isupper())

def num_vowels(gram):
    string = ' '.join(gram)
    vowels = 0
    for i in string:
        if(i.lower() == 'a' or i.lower() == 'e' or i.lower() == 'i' or i.lower() == 'o' or i.lower() == 'u'):
            vowels=vowels+1
    return vowels
           
def num_consonants(gram):
    string = ' '.join(gram)
    cons = list(set('bcdfghjklmnpqrstvwzyx'))
    consonants = 0
    for i in string:
        if i.lower() in cons:
            consonants = consonants + 1
    return consonants
           
def has_digit(gram):
    string = ' '.join(gram)
    found = 0
    for i in string:
        if i.isdigit() is True:
            found = 1
            break
           
    if found == 1:
        return 1
    else:
        return 0

def has_symbols(gram):
    found  = 0
    string = ' '.join(gram)
    symbols = list(set('!,.@_!#$%^&*()<>?/\}{~:]-'))
    for i in string:
        if i in symbols:
            found = 1
            break
           
    if found == 1:
        return 1
    else:
        return 0


def remove_stopWords(one_grams, labels_ones, two_grams, labels_bi, thr_grams, labels_thr, features):
    
    candidates = []
    labels = []
    
    indexes = []
    stop_list = set(stopwords.words('english'))
    for i in range(0, len(one_grams)):
        if one_grams[i].lower() in stop_list:
            indexes.append(i)
    one_grams = [v for i,v in enumerate(one_grams) if i not in frozenset(indexes)]
    labels_ones = [v for i,v in enumerate(labels_ones) if i not in frozenset(indexes)]
    features.drop(indexes, inplace=True)
    features.reset_index(drop=True, inplace=True)
    
    indexes = []
    indexes_feat = []
    for i in range(0, len(two_grams)):
        if two_grams[i][0].lower() in stop_list or two_grams[i][1].lower() in stop_list:
            indexes.append(i)
            # ----- check if previous n-gram is empty now
            if len(one_grams)!= 0:
                indexes_feat.append(len(one_grams)+i-1)
            else:
                indexes_feat.append(i)
            # ------------------------------------------
    two_grams = [v for i,v in enumerate(two_grams) if i not in frozenset(indexes)]
    labels_bi = [v for i,v in enumerate(labels_bi) if i not in frozenset(indexes)]
    features.drop(indexes_feat, inplace=True)
    features.reset_index(drop=True, inplace=True)
    
    indexes = []
    indexes_feat = []
    for i in range(0, len(thr_grams)):
        if thr_grams[i][0].lower() in stop_list or thr_grams[i][1].lower() in stop_list or thr_grams[i][2].lower() in stop_list:
            indexes.append(i)
            indexes_feat.append(len(one_grams)+len(two_grams)+i-1)
    thr_grams = [v for i,v in enumerate(thr_grams) if i not in frozenset(indexes)]
    labels_thr = [v for i,v in enumerate(labels_thr) if i not in frozenset(indexes)] 
    features.drop(indexes_feat, inplace=True)
    features.reset_index(drop=True, inplace=True)
    
    if len(one_grams) != 0:
        for i in range(0, len(one_grams)):
            candidates.append(one_grams[i])
            labels.append(labels_ones[i])
    if len(two_grams) != 0:  
        for i in range(0, len(two_grams)):
            candidates.append(two_grams[i])
            labels.append(labels_bi[i])
    if len(thr_grams) != 0:
        for i in range(0, len(thr_grams)):
            candidates.append(thr_grams[i])
            labels.append(labels_thr[i])
    
    return candidates, labels, features
    
def remove_dupl_train(one_grams, labels_ones, two_grams, labels_bi, thr_grams, labels_thr, sent, features):
    
    
    indexes = []
    indexes_two = []
    for i in range(0, len(one_grams)):
        if len(two_grams) != 0:
            if labels_ones[i] == 0:
                for j in range(0, len(two_grams)):
                    if one_grams[i] == two_grams[j][0] and labels_bi[j] == 1:
                        indexes.append(i)
                    elif one_grams[i] == two_grams[j][1] and labels_bi[j] == 1:
                        indexes.append(i)
        if len(thr_grams) != 0:
            if labels_ones[i] == 0:
                for j in range(0, len(thr_grams)):
                    if one_grams[i] == thr_grams[j][0] and labels_thr[j] == 1:
                        indexes.append(i)
                    elif one_grams[i] == thr_grams[j][1] and labels_thr[j] == 1:
                        indexes.append(i)
                    elif one_grams[i] == thr_grams[j][2] and labels_thr[j] == 1:
                        indexes.append(i)
    for i in range(0, len(two_grams)):
        if len(thr_grams) != 0:
            if labels_bi[i] == 0:
                for j in range(0, len(thr_grams)):
                    if (two_grams[i][0] == thr_grams[j][0] or two_grams[i][0] == thr_grams[j][1]) and labels_thr[j] == 1:
                        indexes_two.append(i)
                    elif (two_grams[i][1] == thr_grams[j][1] or two_grams[i][0] == thr_grams[j][2]) and labels_thr[j] == 1:
                        indexes_two.append(i)

    one_grams = [v for i,v in enumerate(one_grams) if i not in frozenset(indexes)]
    labels_ones = [v for i,v in enumerate(labels_ones) if i not in frozenset(indexes)]
    features.drop(indexes, inplace=True)
    features.reset_index(drop=True, inplace=True)

    two_grams = [v for i,v in enumerate(two_grams) if i not in frozenset(indexes_two)]
    labels_bi = [v for i,v in enumerate(labels_bi) if i not in frozenset(indexes_two)]
    indexes_two = [idx+len(one_grams)-1 for idx in indexes_two]
    features.drop(indexes_two, inplace=True)
    features.reset_index(drop=True, inplace=True)
    return one_grams, labels_ones, two_grams, labels_bi, thr_grams, labels_thr, features

def extract_candidates(tok, sent, pos_tags):
    
    dest = pd.read_csv('destinations.csv', sep=',')
    dest = dest.iloc[:, 1].tolist()
    dest = [w.lower() for w in dest]
    dest = dest + ['hellenic','munich', 'lisburn', 'middlesbrough', 'scot', 'exeter', 'caledonian','ibrox', 'charlton' ,'dutch' ,'african', 'greece', 'greek', 'athens', 'netherlands', 'italy', 'south', 'korea', 'europe','spain', 'france', 'finland', 'sweden', 'asia', 'continental', 'british', 'irishman','britain', 'uk', 'usa', 'london', 'manchester', 'egeland', 'central', 'america', 'north''east', 'west', 'thailand', 'indonesia', 'india', 'sri', "lan", 'world', 'maldives','islands', 'malaysia', 'zurich', 'german', 'swiss', 'hanover', 'western', 'eastern','australia', 'australian', 'cambridge', 'england', 'jamaica', 'jamaican','hill', 'belgrade', 'troy', 'australian', 'las', 'vegas', 'california', 'los', 'angeles''russia', 'russian', 'china', 'italy', 'siberian', 'sudan', 'vietnam', 'indian','new', 'delhi', 'valley', 'moscow', 'boston', 'european', 'albania', 'albanian', 'balkans','atlantic', 'euro',  'macedonia', 'bulgaria', 'sea', 'adriatic', 'vlore','burgas', 'bulgarian', 'macedonian', 'turkish', 'caspioan', 'york','maryland', 'washington', 'county', 'chelsea', 'united', 'zealand', 'paris','miami', 'florida', 'hoolywood', 'denmark', 'angola', 'korea','germany', 'palestinian', 'israel', 'utah', 'french', 'france', 'irish','northwest', 'canada', 'ontario', 'africa', 'jamaica', 'nation', 'nations','international', 'iraq', 'city', 'town', 'country', 'czech', 'republic','portugal', 'ohio', 'river', 'mountain', 'canal', 'canals', 'chicago', 'scadinavian''indonesia', 'egypt', 'earth', 'norway', 'global', 'scotland', 'taiwan','japanese', 'rotterdam', 'scotland', 'sydney', 'houston', 'midlands', 'latin','kingdom', 'mississipi', 'chinese', 'mongolia', 'deutsche', 'central','brixton', 'bristol', 'ireland', 'national', 'cambridge', 'australasia','japan', 'pacific', 'chile', 'federal', 'boulevard', 'world','philadelphia', 'disney', 'venezuela', 'argentina', 'brazil', 'aires', 'buenos','venezuelan', 'dublin', 'frances', 'mexican', 'oklahoma', 'michigan', 'street','edinburgh', 'kenyan', 'moroccan', 'ethiopian', 'swede', 'dutchman', 'briton']
    pd.Series(dest).to_csv('destinations.csv', sep=',')
    
    sport = ['chelsea', 'bayern','sprint', 'live', 'dinamo', 'training', 'grand','juventus','sprinter', 'marathon', 'atletico','footballers','newcastle' ,'anti-doping' , 'champion', 'championship', 'manchester','boxing', 'reebok', 'celtics', 'celtic', 'games', 'game', 'sport', 'sports', 'inter', 'iaaf', 'arena', 'crystal', 'warriors', 'athletics', 'football', 'carling', 'cup', 'fa', 'fifa', 'liverpool','premier', 'premiership', 'league', 'union', 'trafford', 'olympic', 'olympics','olympian', 'palace', 'arsenal', 'arsene', 'tottenham', 'athletico', 'madrid', 'primera', 'liga','daily', 'baros', 'millenium', 'stadium', 'brom', 'reds', 'blues', 'aston', 'villa', 'cross', 'challenge', 'court']
    date = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday', 'january', 'february', 'march','april', 'may', 'june', 'july', 'september', 'october', 'november', 'december', 'day', 'month', 'year','eve', 'christmas']
    prefix = ['mr', 'mrs', 'ms', 'miss', 'mx', 'master', 'maid', 'madam','professor', 'prof', 'chief', 'phd', 'dphil', 'dr', 'doc','advocate', 'economist', 'chancellor', 'lord', 'sir', 'dame','jr', 'duke', 'don', 'president', 'prince', 'king', 'emperor','queen', 'leader', 'emperor', 'empress', 'lord', 'tsar', 'tsarina','count', 'countess', 'duchess', 'vice', 'ambassador', 'secretary','governor', 'mayor', 'councillor', 'speaker', 'doctor', 'representative','senator', 'hon', 'aunt', 'uncle', 'prelate', 'premier', 'Prefect', 'burgess''envoy', 'provost', 'priest','father', 'catholic', 'cardinal', 'dean', 'minister','reader', 'teacher', 'christ', 'major', 'general', 'officer', 'captain', 'engineer','principal', 'coach', 'player', 'citizen', 'male', 'female', 'mother', 'father', 'executive','founder', 'lawyer', 'former', 'challenger', 'laureate', 'pioneer', 'chairman', 'boss', 'member','challenger', 'minister', 'explained', 'boss','member', 'analyst', 'commissioner', 'explains','commented', 'creator', 'director',  'co-chairman', 'former', 'manager', 'who', 'designer','told', 'star', 'writer', 'ruled', 'actor', 'spokesman', 'nominee', 'actress', 'daughter','winner', 'artist', 'anchor', 'model', 'comedian', 'spokeswoman', 'singer','member', 'prop']
    f0, f1, f2, f3, f4, f5, f6, f7, f8, f9, f10, f11, f12, f13, f14, f15, f16, f17 = ([] for i in range(18))
    
    one_grams = []
    two_grams = []
    thr_grams = []
    
    labels_ones= []
    labels_bi = []
    labels_thr = []
    
    for i in range(0, len(tok)):
        if str.isupper(tok[i][0]) is True:
            one_grams.append(tok[i])
            # ------create feautures --------
            f0.append(1)
            f1.append(0)
            f2.append(0)
            f3.append(feature_noun(pos_tags[i]))
            f4.append(feature_adjective(pos_tags[i]))
            f5.append(feature_verb(pos_tags[i]))
            f6.append(end_sent(i))
            
            if i != len(tok)-1 and tok[i+1] == "'s":
                f7.append(1)
            elif i != len(tok)-4 and tok[i+1] == "<" and tok[i+4] == "'s":
                f7.append(1)
            else:
                f7.append(0)
            
            if i != 0 and tok[i-1] in prefix:
                f8.append(1)
            elif i != 0 and tok[i-1] == ">" and i-4>=0 and tok[i-4] in prefix:
                f8.append(1)
            else:
                f8.append(0)
            
            f9.append(feat_specific(dest, tok[i]))
            f10.append(feat_specific(date, tok[i]))
            f11.append(num_capitals(tok[i]))
            f12.append(num_vowels(tok[i]))
            f13.append(num_consonants(tok[i]))
            f14.append(has_symbols(tok[i]))
            f15.append(has_digit(tok[i]))
            
            verbs = ["VB", "VBD", "VBG", "VBN", "VBP", "VBZ"]
            nouns = ["NN", "NNP"]
            if i != len(tok)-1 and pos_tags[i+1][1] in verbs and pos_tags[i][1] in nouns:
                f16.append(1)
            elif i != len(tok)-2 and pos_tags[i+1][1] == 'TO' and pos_tags[i+2][1] in verbs and pos_tags[i][1] in nouns:
                f16.append(1)
            elif i != len(tok)-2 and pos_tags[i+1][1] == 'MD' and pos_tags[i+2][1] in verbs and pos_tags[i][1] in nouns:
                f16.append(1)
            elif i != len(tok)-4 and tok[i+1] == '<' and pos_tags[i][1] in nouns and pos_tags[i+4][1] in verbs:
                f16.append(1)
            elif i != len(tok)-5 and tok[i+1] == '<' and pos_tags[i+4][1] == 'TO' and pos_tags[i+5][1] in verbs and pos_tags[i][1] in nouns:
                f16.append(1)
            elif i != len(tok)-5 and tok[i+1] == '<' and pos_tags[i+4][1] == 'MD' and pos_tags[i+5][1] in verbs and pos_tags[i][1] in nouns:
                f16.append(1)
            else:
                f16.append(0)
            f17.append(feat_specific(sport, tok[i]))
            # -------------------------------
            if i!=0 and tok[i-1] == '>' and tok[i+1] == '<':
                labels_ones.append(1)
            else:
                labels_ones.append(0)
            
    bigrams = list(ngrams(tok, 2))
    for i in range(0, len(bigrams)):
        if str.isupper(bigrams[i][0][0]) is True and str.isupper(bigrams[i][1][0]) is True:
            two_grams.append(bigrams[i])
            
            # ------create feautures --------
            f0.append(0)
            f1.append(1)
            f2.append(0)
            for m in range(0, len(tok)):
                if m != len(tok)-1:
                    if tok[m] == bigrams[i][0] and tok[m+1] == bigrams[i][1]:
                        f3.append(feature_noun(pos_tags[m]))
                        f4.append(feature_adjective(pos_tags[m]))
                        f5.append(feature_verb(pos_tags[m]))
                        f6.append(end_sent(m))
                        
                        if i != len(tok)-2 and tok[i+2] == "'s":
                            f7.append(1)
                        elif i != len(tok)-5 and tok[i+2] == "<" and tok[i+5] == "'s":
                            f7.append(1)
                        else:
                            f7.append(0)
                            
                        if i != 0 and tok[i-1] in prefix:
                            f8.append(1)
                        elif i != 0 and tok[i-1] == ">" and i-4>=0 and tok[i-4] in prefix:
                            f8.append(1)
                        else:
                            f8.append(0)
                        
                        f9.append(feat_specific(dest, bigrams[i]))
                        f10.append(feat_specific(date, bigrams[i]))
                        f11.append(num_capitals(bigrams[i]))
                        f12.append(num_vowels(bigrams[i]))
                        f13.append(num_consonants(bigrams[i]))
                        f14.append(has_symbols(bigrams[i]))
                        f15.append(has_digit(bigrams[i]))
                        
                        verbs = ["VB", "VBD", "VBG", "VBN", "VBP", "VBZ"]
                        nouns = ["NN", "NNP"]
                        if i != len(tok)-2 and pos_tags[i+2][1] in verbs and pos_tags[i][1] in nouns:
                            f16.append(1)
                        elif i != len(tok)-3 and pos_tags[i+2][1] == 'TO' and pos_tags[i+3][1] in verbs and pos_tags[i][1] in nouns:
                            f16.append(1)
                        elif i != len(tok)-3 and pos_tags[i+2][1] == 'MD' and pos_tags[i+3][1] in verbs and pos_tags[i][1] in nouns:
                            f16.append(1)
                        elif i != len(tok)-5 and tok[i+2] == '<' and pos_tags[i][1] in nouns and pos_tags[i+5][1] in verbs:
                            f16.append(1)
                        elif i != len(tok)-6 and tok[i+2] == '<' and pos_tags[i+5][1] == 'TO' and pos_tags[i+6][1] in verbs and pos_tags[i][1] in nouns:
                            f16.append(1)
                        elif i != len(tok)-6 and tok[i+2] == '<' and pos_tags[i+5][1] == 'MD' and pos_tags[i+6][1] in verbs and pos_tags[i][1] in nouns:
                            f16.append(1)
                        else:
                            f16.append(0)
                        
                        f17.append(feat_specific(sport, bigrams[i]))
                        
                        break
            
            # -------------------------------

            if i!=0 and tok[i-1] == '>' and tok[i+2] == '<':
                labels_bi.append(1)
            else:
                labels_bi.append(0)
                
    threegrams = list(ngrams(tok, 3))
    for i in range(0, len(threegrams)):
        if str.isupper(threegrams[i][0][0]) is True and str.isupper(threegrams[i][1][0]) is True and str.isupper(threegrams[i][2][0]) is True:
            thr_grams.append(threegrams[i])
            
            # ------create feautures --------
            f0.append(0)
            f1.append(0)
            f2.append(1)
            for m in range(0, len(tok)):
                if m != len(tok)-2:
                    if tok[m] == threegrams[i][0] and tok[m+1] == threegrams[i][1] and tok[m+2] == threegrams[i][2]:
                        f3.append(feature_noun(pos_tags[m]))
                        f4.append(feature_adjective(pos_tags[m]))
                        f5.append(feature_verb(pos_tags[m]))
                        f6.append(end_sent(m))
                        if i != len(tok)-3 and tok[i+3] == "'s":
                            f7.append(1)
                        elif i != len(tok)-6 and tok[i+3] == "<" and tok[i+6] == "'s":
                            f7.append(1)
                        else:
                            f7.append(0)
                        
                        if i != 0 and tok[i-1] in prefix:
                            f8.append(1)
                        elif i != 0 and tok[i-1] == ">" and i-4>=0 and tok[i-4] in prefix:
                            f8.append(1)
                        else:
                            f8.append(0)
                        f9.append(feat_specific(dest, threegrams[i])) 
                        f10.append(feat_specific(date, threegrams[i]))
                        f11.append(num_capitals(threegrams[i]))
                        f12.append(num_vowels(threegrams[i]))
                        f13.append(num_consonants(threegrams[i]))
                        f14.append(has_symbols(threegrams[i]))
                        f15.append(has_digit(threegrams[i]))
                        
                        
                        if i != len(tok)-3 and pos_tags[i+3][1] in verbs and pos_tags[i][1] in nouns:
                            f16.append(1)
                        elif i != len(tok)-4 and pos_tags[i+3][1] == 'TO' and pos_tags[i+4][1] in verbs and pos_tags[i][1] in nouns:
                            f16.append(1)
                        elif i != len(tok)-4 and pos_tags[i+3][1] == 'MD' and pos_tags[i+4][1] in verbs and pos_tags[i][1] in nouns:
                            f16.append(1)
                        elif i != len(tok)-6 and tok[i+3] == '<' and pos_tags[i][1] in nouns and pos_tags[i+6][1] in verbs:
                            f16.append(1)
                        elif i != len(tok)-7 and tok[i+3] == '<' and pos_tags[i+5][1] == 'TO' and pos_tags[i+7][1] in verbs and pos_tags[i][1] in nouns:
                            f16.append(1)
                        elif i != len(tok)-7 and tok[i+3] == '<' and pos_tags[i+5][1] == 'MD' and pos_tags[i+7][1] in verbs and pos_tags[i][1] in nouns:
                            f16.append(1)
                        else:
                            f16.append(0)
                            
                        f17.append(feat_specific(sport, threegrams[i]))
                        break
            # -------------------------------
            
            if i!=0 and tok[i-1] == '>' and tok[i+3] == '<':
                labels_thr.append(1)
            else:
                labels_thr.append(0)

    
    features = pd.DataFrame({'f0': f0})
    features['f1'] = pd.Series(f1, index=features.index)
    features['f2'] = pd.Series(f2, index=features.index)
    features['f3'] = pd.Series(f3, index=features.index)
    features['f4'] = pd.Series(f4, index=features.index)
    features['f5'] = pd.Series(f5, index=features.index)
    features['f6'] = pd.Series(f6, index=features.index)
    features['f7'] = pd.Series(f7, index=features.index)
    features['f8'] = pd.Series(f8, index=features.index)
    features['f9'] = pd.Series(f9, index=features.index)
    features['f10'] = pd.Series(f10, index=features.index)
    features['f11'] = pd.Series(f11, index=features.index)
    features['f12'] = pd.Series(f12, index=features.index)
    features['f13'] = pd.Series(f13, index=features.index)
    features['f14'] = pd.Series(f14, index=features.index)
    features['f15'] = pd.Series(f15, index=features.index)
    features['f16'] = pd.Series(f16, index=features.index)
    features['f17'] = pd.Series(f17, index=features.index)
    
    one_grams, labels_ones, two_grams, labels_bi, thr_grams, labels_thr, features = remove_dupl_train(one_grams, labels_ones, two_grams, labels_bi, thr_grams, labels_thr, sent, features)
    sent_cand, labels, features = remove_stopWords(one_grams, labels_ones, two_grams, labels_bi, thr_grams, labels_thr, features)

    return sent_cand, labels, features


def process_sentence(sent):
    
    tokens = nltk.word_tokenize(sent)
    pos_tags = nltk.pos_tag(tokens)
    sent_cand, labels, sent_features = extract_candidates(tokens, sent, pos_tags)
    return sent_cand, labels, sent_features
    

def read_files(type_file):
    path = "/Users/elenamilkai/Desktop/project-stage1/data/"+type_file+'Y/'
    #path = "/Users/elenamilkai/Desktop/project-stage1/data/"+type_file+'/'
    candidates = []
    labels_all = []
    features_all = pd.DataFrame(columns=['f0','f1','f2','f3'])
    
    num_files = 0
    for filename in os.listdir(path):
        if not filename.startswith('.'):
            num_files += 1
            print('--------File--------> '+str(num_files)+'\n')
            with open(path + filename, 'r') as myfile:
                data = myfile.read().replace('\n', ' ')
                sentences = nltk.tokenize.sent_tokenize(data)
               
                for j in range(0, len(sentences)):
                    cand_sent, labels, sent_feat = process_sentence(sentences[j])
                    features_all = features_all.append(sent_feat)
                    if len(cand_sent) != 0:
                        for k in range(0, len(cand_sent)):
                            candidates.append(cand_sent[k])
                            labels_all.append(labels[k])
    
    features_all.reset_index(drop=True, inplace=True)
    pd.Series(labels_all).to_csv('results/'+type_file+'/labels.csv', sep=',')
    pd.Series(candidates).to_csv('results/'+type_file+'/candidates.csv', sep=',')
    features_all.to_csv('results/'+type_file+'/features.csv', sep=',')


if __name__ == '__main__':
    print('Start with training data in set I.\n')
    read_files('train') #debug/Q
    print('Continue with testing data in set J.\n')
    read_files('test') 
    print('Finish!')

Start with training data in set I.

--------File--------> 1

--------File--------> 2

--------File--------> 3

--------File--------> 4

--------File--------> 5

--------File--------> 6

--------File--------> 7

--------File--------> 8

--------File--------> 9

--------File--------> 10

--------File--------> 11

--------File--------> 12

--------File--------> 13

--------File--------> 14

--------File--------> 15

--------File--------> 16

--------File--------> 17

--------File--------> 18

--------File--------> 19

--------File--------> 20

--------File--------> 21

--------File--------> 22

--------File--------> 23

--------File--------> 24

--------File--------> 25

--------File--------> 26

--------File--------> 27

--------File--------> 28

--------File--------> 29

--------File--------> 30

--------File--------> 31

--------File--------> 32

--------File--------> 33

--------File--------> 34

--------File--------> 35

--------File--------> 36

--------File--------> 37

--------Fil